In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
cred = pd.read_parquet('../data/credit_history.parquet', engine="fastparquet")
xml = pd.read_xml('../data/geographic_data.xml')
xlsx = pd.read_excel('../data/loan_details.xlsx')
jsonl = pd.read_json('../data/financial_ratios.jsonl', lines=True)
demog = pd.read_csv('../data/demographics.csv')
appl = pd.read_csv('../data/application_metadata.csv')

In [33]:
dfs = [cred, xml, xlsx, jsonl, demog, appl]
key_names = ["customer_number", "id", "customer_id", "cust_num", "cust_id", "customer_ref"]

for i in range(len(dfs)):
    dfs[i] = dfs[i].rename(columns={key_names[i]: "customer_id"})

In [34]:
from functools import reduce

merged = reduce(
    lambda left, right: left.merge(right, on="customer_id", how="inner"),
    dfs
)

In [36]:
merged.to_csv('../data/merged_data.csv', index=False)

In [38]:
df = pd.read_csv('../data/merged_data.csv')

In [ ]:
pd.set_option("display.max_columns", None)

# loan_officer_id, 

In [ ]:
df

,customer_id,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,num_delinquencies_2yrs,num_inquiries_6mo,recent_inquiry_count,num_public_records,num_collections,account_diversity_index,state,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign,monthly_income,existing_monthly_debt,monthly_payment,debt_to_income_ratio,debt_service_ratio,payment_to_income_ratio,credit_utilization,revolving_balance,credit_usage_amount,available_credit,total_monthly_debt_payment,annual_debt_payment,loan_to_annual_income,total_debt_amount,monthly_free_cash_flow,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents,application_id,application_hour,application_day_of_week,account_open_year,preferred_contact,referral_code,account_status_code,random_noise_1,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing,default
0,10000,696,14,22.8,273.6,169100.0,0.0,2,2,1,0,0.499,OH,4.8,56000,1380.0,91.0,73.0,451,Personal,"$17,700",36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W,"5,150.00",738.64,$592.13,0.258,0.258402,0.115,0.841,"$142,213.10","$142,213.10","$26,886.90",1330.77,15969.24,0.286408,"159,913.10","3,819.23",41,$61800,2.2,Full-time,Graduate,Married,2,620515,5,6,2013,Mail,REF0000,ACT-2,1.137099,13,2,1,1,0
1,10001,659,13,3.5,42.0,78200.0,0.0,6,6,0,0,0.298,PA,4.4,61000,1510.0,92.0,87.0,537,mortgage,"$114,000",180,6.83,Refinance,0.774,Branch,1011,B,"2,383.33",392.21,"1,013.86",0.590,0.589959,0.425,0.971,"$75,932.20","75,932.20","$2,267.80",1406.07,16872.84,3.986014,"189,932.20",$977.26,38,"28,600",7.0,FULL_TIME,High School,Married,0,624978,4,2,2015,Phone,REF0000,ACT-3,-0.164932,6,1,1,1,1
2,10002,662,3,0.0,0.0,41400.0,0.0,2,2,0,0,0.174,VA,3.9,74000,1920.0,125.0,103.0,679,Personal Loan,"9,300",36,13.99,Major Purchase,0.000,Online,1084,K,"1,725.00",204.07,$317.81,0.303,0.302539,0.184,0.539,22314.6,"22,314.60","$19,085.40",521.88,6262.56,0.449275,31614.6,1203.12,18,"$20,700",0.8,FULL_TIME,Bachelor,Single,0,564658,10,3,2020,Phone,REF0000,ACT-3,0.526700,1,2,1,0,0
3,10003,676,8,9.0,108.0,60000.0,0.0,1,1,0,0,0.263,CA,5.8,75000,1690.0,158.0,121.0,719,PERSONAL,$8700,48,13.26,Medical,0.000,Online,1048,A,"2,616.67",$288.71,234.52,0.200,0.199961,0.090,0.147,"8,820.00",8820.0,"51,180.00",523.23,6278.76,0.277070,"$17,520.00",2093.4366666666665,27,"31,400",4.8,Full Time,Bachelor,Single,0,621493,7,5,2010,Email,REF0000,A01,-0.709779,4,1,1,1,0
4,10004,678,7,8.0,96.0,49700.0,0.0,1,1,0,0,0.298,WA,5.8,78000,1700.0,152.0,127.0,933,Personal,"$7,200",24,10.77,Debt Consolidation,0.000,Branch,1055,S,"2,050.00",$248.77,334.81,0.285,0.284673,0.163,0.488,24253.6,24253.6,"25,446.40",583.58,7002.96,0.292683,"$31,453.60",1466.42,26,$24600,5.2,Fulltime,High School,Single,0,637785,1,2,2020,Mail,REF0000,ACT-3,-0.603132,6,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89994,99994,817,10,8.2,98.4,135600.0,0.0,1,1,0,0,0.285,CA,5.8,75000,1830.0,168.0,138.0,933,personal,"$11,300",24,9.70,Home Improvement,0.000,Branch,1050,O,"6,025.00","2,120.97",519.87,0.438,0.438314,0.086,0.359,"48,680.40","48,680.40","$86,919.60","2,640.84",31690.08,0.156293,"$59,980.40","$3,384.16",52,"$72,300",3.7,Full-time,Bachelor,Single,1,585752,13,5,2016,Email,REF0000,A01,1.628917,10,3,0,1,0
89995,99995,745,9,8.8,105.6,44600.0,0.0,1,1,0,0,0.353,IL,4.9,65000,1900.0,99.0,84.0,536,personal,"16,500",36,11.67,Other,0.000,Branch,1071,D,"$1,666.67",138.97,$545.44,0.411,0.410646,0.327,0.355,"15,833.00",15833.0,"$28,767.00",$684.41,8212.92,0.825000,32333.0,982.26,61,"20,000",4.2,Full-time,High S

In [49]:
df['marketing_campaign'].value_counts()

marketing_campaign
Q    3549
N    3541
O    3535
J    3522
E    3511
X    3503
I    3492
M    3486
Z    3485
H    3485
K    3468
A    3463
B    3460
G    3459
V    3458
T    3453
D    3452
C    3451
S    3446
Y    3434
F    3429
U    3420
P    3399
R    3386
W    3369
L    3343
Name: count, dtype: int64